In [10]:
import glob
import os
import pandas as pd

csv_files = glob.glob("ativos/*.csv")

# Vamos montar uma lista de dicionários com Ticker e Origem
resultados = []

for fn in csv_files:
    nome_arquivo = os.path.basename(fn)
    df = pd.read_csv(fn, header=None, dtype=str)

    # separa em dois pares (Ticker, Cor) conforme antes
    df1 = df[[0, 2]].rename(columns={0: "Ticker", 2: "Cor"})
    df2 = df[[5, 7]].rename(columns={5: "Ticker", 7: "Cor"})
    df_all = pd.concat([df1, df2], ignore_index=True)

    df_all["Ticker"] = df_all["Ticker"].str.strip()
    df_all["Cor"]    = df_all["Cor"].str.strip().str.lower()

    # filtra só os verdes e registra a origem
    verdes = df_all.loc[df_all["Cor"] == "vermelho", "Ticker"]
    for ticker in verdes:
        resultados.append({
            "Ticker": ticker,
            "Setor": nome_arquivo
        })

# transforma em DataFrame para visualizar
df_res = pd.DataFrame(resultados)
print(df_res)


    Ticker                                Setor
0      EDU  Academic & Educational Services.csv
1      TAL  Academic & Educational Services.csv
2      GHC  Academic & Educational Services.csv
3     AFYA  Academic & Educational Services.csv
4     GOTU  Academic & Educational Services.csv
..     ...                                  ...
179    RCI                       Technology.csv
180    EBR                        Utilities.csv
181    BEP                        Utilities.csv
182    BEP                        Utilities.csv
183    EBR                        Utilities.csv

[184 rows x 2 columns]


In [4]:
import glob
import pandas as pd

csv_files = glob.glob("ativos/*.csv")

verdes = []
amarelos = []
vermelhos = []

for fn in csv_files:
    # 1) Lê pulando a primeira linha (header) e pega só as 2 colunas necessárias
    df = pd.read_csv(
        fn,
        header=None,          # diz que não vamos usar o cabeçalho do arquivo
        skiprows=1,           # pula a linha que contém TAMAN,SCORE,COR
        usecols=[0,1],        # coluna 0 = ticker, coluna 1 = score
        names=["Ticker","Score"],  # nomes das colunas no DataFrame
        decimal=","           # para converter "38,24" → 38.24
    )
    
    # 2) Converte Score para float (já cuida da vírgula)
    df["Score"] = df["Score"].astype(float)
    
    # 3) Classifica em vermelho/amarelo/verde
    df["Cor"] = pd.cut(
        df["Score"],
        bins=[-float("inf"), 40, 60, float("inf")],
        labels=["vermelho","amarelo","verde"]
    )
    
    # 4) Acumula em cada lista
    verdes   .extend(df.loc[df["Cor"]=="verde",    "Ticker"].tolist())
    amarelos .extend(df.loc[df["Cor"]=="amarelo",  "Ticker"].tolist())
    vermelhos.extend(df.loc[df["Cor"]=="vermelho", "Ticker"].tolist())

# 5) (Opcional) Remove duplicatas mantendo ordem
verdes    = list(dict.fromkeys(verdes))
amarelos  = list(dict.fromkeys(amarelos))
vermelhos = list(dict.fromkeys(vermelhos))

print("VERDES   :", verdes)
print("AMARELOS :", amarelos)
print("VERMELHOS:", vermelhos)


VERDES   : ['IP', 'MCD', 'LOW', 'TJX', 'CMG', 'RACE', 'ORLY', 'YUM', 'PM', 'MNST', 'AON', 'MCK', 'WM', 'RSG', 'CARR', 'GWW', 'PAYX', 'FAST', 'WAB', 'LUV', 'EQIX', 'O', 'BEKE', 'INVH', 'V', 'CRM', 'TMUS', 'IBM', 'ADI', 'APH']
AMARELOS : ['LRN', 'LOPE', 'ATGE', 'LAUR', 'STRA', 'PRDO', 'UTI', 'FC', 'APEI', 'VSTA', 'RIO', 'SHW', 'APD', 'SCCO', 'ECL', 'FCX', 'AEM', 'CTVA', 'VMC', 'MLM', 'PPG', 'NTR', 'LYB', 'PKG', 'MT', 'STLD', 'BALL', 'RS', 'AVY', 'WLK', 'GFI', 'JHX', 'AMCR', 'AU', 'AMZN', 'TSLA', 'COST', 'HD', 'DIS', 'BKNG', 'SBUX', 'NKE', 'MAR', 'RCL', 'TGT', 'HLT', 'AZO', 'LULU', 'ROST', 'TCOM', 'F', 'CUK', 'CCL', 'LYV', 'DECK', 'LVS', 'TSCO', 'WSM', 'NVR', 'BRK-A', 'BRK-B', 'PG', 'KO', 'PEP', 'HON', 'BUD', 'MMM', 'MDLZ', 'CL', 'KR', 'KMB', 'KDP', 'KVUE', 'CCEP', 'XOM', 'CVX', 'SHEL', 'COP', 'TTE', 'ENB', 'EOG', 'EPD', 'ET', 'WMB', 'KMI', 'SLB', 'OKE', 'MPLX', 'LNG', 'FANG', 'SU', 'OXY', 'HES', 'BAC', 'WFC', 'MS', 'AXP', 'GS', 'RY', 'BLK', 'SCHW', 'PGR', 'CB', 'MMC', 'APO', 'ICE', 'NWG'

In [12]:
import glob
import os
import pandas as pd

# 1) Liste todos os CSVs dentro de ativos/
csv_files = glob.glob("ativos/*.csv")

# 2) Acumula todos os registros aqui
dados = []

for fn in csv_files:
    # captura o nome do CSV atual
    nome_arquivo = os.path.basename(fn)
    
    df = pd.read_csv(
        fn,
        header=None,
        skiprows=1,            # pula a linha de cabeçalho original
        usecols=[0,1],         # coluna 0 = Ticker, 1 = Score
        names=["Ticker","Score"],
        decimal=","            # converte "38,24" → 38.24
    )
    # 3) Classificação por cor
    df["Cor"] = pd.cut(
        df["Score"],
        bins=[-float("inf"), 40, 60, float("inf")],
        labels=["vermelho","amarelo","verde"]
    )
    # 4) Adiciona linha a linha na lista, agora com o setor correto
    for _, row in df.iterrows():
        dados.append({
            "Ticker": row["Ticker"].strip(),
            "Score":  row["Score"],
            "Cor":    row["Cor"],
            "Setor":  nome_arquivo
        })

# 5) Converte para DataFrame único
df_todos = pd.DataFrame(dados)

# 6) Ordena decrescente por Score
df_todos = df_todos.sort_values("Score", ascending=False).reset_index(drop=True)

# 7) Exibe ou exporta
print(df_todos)
df_todos.to_csv("ativos_classificados_ordenados.csv", index=False)


    Ticker  Score       Cor                                Setor
0      CRM  73.88     verde                       Technology.csv
1     RACE  70.87     verde               Consumer Cyclicals.csv
2     CARR  69.16     verde                        INDUSTRIA.csv
3      YUM  67.83     verde               Consumer Cyclicals.csv
4     EQIX  67.49     verde                      Real Estate.csv
..     ...    ...       ...                                  ...
348   FEDU  19.00  vermelho  Academic & Educational Services.csv
349    ADD  17.00  vermelho  Academic & Educational Services.csv
350     JZ  17.00  vermelho  Academic & Educational Services.csv
351   GSUN  16.50  vermelho  Academic & Educational Services.csv
352    COE  14.50  vermelho  Academic & Educational Services.csv

[353 rows x 4 columns]


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 1) Carrega o arquivo exportado anteriormente
df_todos = pd.read_csv("ativos_classificados_ordenados.csv")

# 2) Filtra apenas os tickers "verdes"
list_symbols = df_todos.loc[df_todos["Cor"] == "verde", "Ticker"].tolist()

# 3) Define datas para cálculo de retorno
hoje = datetime.today().date()
um_ano_atras = hoje - relativedelta(years=1)

# 4) Acumula resultados
resultados = []
for ticker in list_symbols:
    tk = yf.Ticker(ticker)
    hist = tk.history(start=um_ano_atras, end=hoje, interval="1d")
    if hist.empty or len(hist) < 2:
        continue

    p0 = hist["Close"].iloc[0]
    p1 = hist["Close"].iloc[-1]
    ganho = (p1 / p0) - 1

    # Classificação de perfil
    if ganho < 0.12:
        perfil = "conservador"
    elif ganho < 0.20:
        perfil = "moderado"
    else:
        perfil = "agressivo"

    resultados.append({
        "Ticker": ticker,
        
        "Preço há 1 ano": round(p0, 2),
        "Preço atual": round(p1, 2),
        "Retorno (%)": round(ganho * 100, 2),
        "Perfil de Invest.": perfil
    })

# 5) Cria DataFrame final e ordena
df_perfis = (
    pd.DataFrame(resultados)
      .sort_values("Retorno (%)", ascending=False)
      .reset_index(drop=True)
)

df_perfis


,Ticker,Preço há 1 ano,Preço atual,Retorno (%),Perfil de Invest.
0,APH,61.55,106.51,73.04,agressivo
1,PM,113.02,164.05,45.14,agressivo
2,FAST,32.47,46.13,42.08,agressivo
3,IBM,184.09,253.15,37.52,agressivo
4,BEKE,13.40,18.42,37.46,agressivo
5,V,264.01,345.47,30.85,agressivo
6,ORLY,75.56,98.32,30.12,agressivo
7,TMUS,184.49,238.41,29.22,agressivo
8,WAB,155.95,192.05,23.15,agressivo
9,LUV,25.76,30.93,20.07,agressivo


In [7]:


import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 1) Carrega o CSV com Score e Cor
df_todos = pd.read_csv("ativos_classificados_ordenados.csv")

# 2) Pega apenas os tickers "verdes"
list_symbols = df_todos.loc[df_todos["Cor"] == "verde", "Ticker"].tolist()

# 3) Datas de 1 ano atrás até hoje
hoje = datetime.today().date()
um_ano_atras = hoje - relativedelta(years=1)

# 4) Monta resultados
resultados = []
for ticker in list_symbols:
    tk = yf.Ticker(ticker)
    hist = tk.history(start=um_ano_atras, end=hoje, interval="1d")
    if hist.empty or len(hist) < 200:
        continue

    p0 = hist["Close"].iloc[0]
    p1 = hist["Close"].iloc[-1]
    ganho = (p1 / p0) - 1

    # Calcula EMAs
    hist["EMA20"]  = hist["Close"].ewm(span=20, adjust=False).mean()
    hist["EMA200"] = hist["Close"].ewm(span=200, adjust=False).mean()
    ema20 = hist["EMA20"].iloc[-1]
    ema200 = hist["EMA200"].iloc[-1]

    # Classifica perfil
    if ganho < 0.12:
        perfil = "conservador"
    elif ganho < 0.20:
        perfil = "moderado"
    else:
        perfil = "agressivo"

    score_val = df_todos.loc[df_todos["Ticker"] == ticker, "Score"].iloc[0]

    resultados.append({
        "Ticker":            ticker,
        "Preço atual":       round(p1, 2),
        "Entrada (EMA20)":   round(ema20, 2),
        "Saída (EMA200)":    round(ema200, 2),
        "Score":             round(score_val, 2),
        "Retorno (%)":       round(ganho * 100, 2),
        "Perfil":            perfil
    })

# 5) DataFrame final e ordenação
df_final = pd.DataFrame(resultados)
df_final = df_final.sort_values("Retorno (%)", ascending=False).reset_index(drop=True)

# Exibe no Jupyter
from IPython.display import display
display(df_final)


,Ticker,Preço atual,Entrada (EMA20),Saída (EMA200),Score,Retorno (%),Perfil
0,APH,106.51,102.11,79.86,60.14,73.04,agressivo
1,PM,164.05,170.18,153.62,60.77,45.14,agressivo
2,FAST,46.13,45.39,39.43,65.58,42.08,agressivo
3,IBM,253.15,273.03,247.23,60.89,37.52,agressivo
4,BEKE,18.42,18.88,18.75,61.99,37.46,agressivo
5,V,345.47,352.16,332.64,65.73,30.85,agressivo
6,ORLY,98.32,94.99,87.87,63.65,30.12,agressivo
7,TMUS,238.41,235.95,233.30,62.88,29.22,agressivo
8,WAB,192.05,203.34,191.63,64.76,23.15,agressivo
9,LUV,30.93,33.85,31.07,60.20,20.07,agressivo


In [ ]:
import yfinance as yf

tickers = ["TLT", "IEF", "SHY"]
hist = yf.download(tickers, start="2020-01-01", end="2025-07-31")
# hist['Close'] é um DataFrame com colunas TLT, IEF, SHY
print(hist['Close'].tail())


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12720\3467669717.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  hist = yf.download(tickers, start="2020-01-01", end="2025-07-31")
[*********************100%***********************]  3 of 3 completed

Ticker            IEF        SHY        TLT
Date                                       
2025-07-24  94.519997  82.500000  85.889999
2025-07-25  94.739998  82.550003  86.430000
2025-07-28  94.529999  82.540001  85.870003
2025-07-29  95.150002  82.610001  87.320000
2025-07-30  94.820000  82.519997  86.870003


In [3]:
import requests
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("FMP_API_KEY")
if not API_KEY:
    raise RuntimeError("FMP_API_KEY não encontrada no .env")
ISIN    = "US718507BK18"

# 1) Busca pelo ISIN para descobrir o symbol interno e o nome do emissor
url_isin = (
    f"https://financialmodelingprep.com/api/v3/search-isin"
    f"?isin={ISIN}&apikey={API_KEY}"
)
resp_isin = requests.get(url_isin)
resp_isin.raise_for_status()
data_isin = resp_isin.json()
if not data_isin:
    raise ValueError(f"ISIN {ISIN} não encontrado na FMP")
      
sec = data_isin[0]
symbol = sec.get("symbol")       # ex.: 'PRI' ou algo equivalente
issuer = sec.get("name")         # nome da empresa emissora

# 2) Puxa as notas da companhia para esse símbolo
url_notes = (
    f"https://financialmodelingprep.com/api/v3/company-notes"
    f"?symbol={symbol}&apikey={API_KEY}"
)
resp_notes = requests.get(url_notes)
resp_notes.raise_for_status()
notes = resp_notes.json()

# 3) Filtra a nota que contenha o nosso ISIN ou CUSIP
#    (muitas vezes o título da nota inclui o CUSIP)
bond_note = next(
    (n for n in notes 
     if ISIN in n.get("title","") or sec.get("cusip") in n.get("title","")),
    None
)
if bond_note is None:
    raise ValueError(f"Nenhuma nota encontrada para ISIN {ISIN}")

# 4) Extrai os dados que interessam
maturity      = bond_note.get("maturityDate")      # ex.: "2028-05-15"
coupon_rate   = bond_note.get("interestRate")      # ex.: 5.25  (em %)
freq_per_year = bond_note.get("paymentFrequency")  # ex.: 2  (semestral)

# 5) Converte frequência em meses entre pagamentos
meses_pagto = 12 // freq_per_year if freq_per_year else None

# 6) Monta um DataFrame resumido
df = pd.DataFrame([{
    "ISIN":                  ISIN,
    "Emissor":               issuer,
    "Vencimento":            maturity,
    "Cupom (%)":             coupon_rate,
    "Freq. Pagto/ano":       freq_per_year,
    "Meses entre pagamentos": meses_pagto
}])

print(df)


ValueError: ISIN US718507BK18 não encontrado na FMP

In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
from pandas_datareader.data import DataReader

# 1) Defina os ETFs de setor (NYSE) como proxy
SECTOR_ETFS = {
    "Consumer Discretionary": "XLY",
    "Consumer Staples":       "XLP",
    "Industrials":            "XLI",
    "Financials":             "XLF",
    "Materials":              "XLB",
    "Energy":                 "XLE",
    "Health Care":            "XLV",
    "Utilities":              "XLU",
    "Technology":             "XLK"
}
BENCH = "SPY"

# 2) Datas
hoje         = datetime.today().date()
seis_meses   = hoje - relativedelta(months=6)

# 3) Busca retornos acumulados 6M
def ret(symbol):
    df = yf.download(symbol, start=seis_meses, end=hoje, progress=False)["Adj Close"]
    return df.pct_change().add(1).cumprod().iloc[-1] - 1

# 4) Monta DataFrame com retornos
rets = {name: ret(ticker) for name,ticker in SECTOR_ETFS.items()}
rets["Benchmark"] = ret(BENCH)
df_rets = pd.Series(rets).to_frame("Retorno6M").sort_values("Retorno6M", ascending=False)

# 5) Fetch ISM PMI e yield curve do FRED
pmi = DataReader("PMI", "fred", seis_meses, hoje).iloc[-1,0]        # ISM Manufacturing PMI
y10 = DataReader("DGS10","fred", seis_meses, hoje).iloc[-1,0]      # 10-yr Treasury
y2  = DataReader("DGS2", "fred", seis_meses, hoje).iloc[-1,0]      # 2-yr Treasury
yield_curve = y10 - y2

# 6) Decide fase de ciclo
if pmi > 50 and df_rets.loc["Technology","Retorno6M"] > df_rets.loc["Benchmark","Retorno6M"]:
    fase = "Expansão"
elif yield_curve < 0:
    fase = "Pico"
elif pmi < 50 and df_rets.loc["Utilities","Retorno6M"] > df_rets.loc["Benchmark","Retorno6M"]:
    fase = "Contração"
else:
    fase = "Fundo"

# 7) Associa cada setor à fase
df_rets["Fase de Ciclo"] = fase

# 8) Exibe
print(f"Indicadores: PMI={pmi:.1f}, YieldCurve={yield_curve:.2f}")
print(df_rets)


ModuleNotFoundError: No module named 'distutils'